In [1]:
# 📦 Стандартные библиотеки
import os
import csv
import json
import random
import datetime
from copy import deepcopy

# 🧰 Прогресс-бары
from tqdm import tqdm

# 📊 Научные библиотеки
import numpy as np
import matplotlib.pyplot as plt

# 🧮 Машинное обучение (метрики, кросс-валидация)
from sklearn.model_selection import KFold
from sklearn.metrics import (
    r2_score,
    mean_absolute_error,
    mean_squared_error
)

# 🔥 PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F

# 🧠 PyTorch Geometric
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import (
    GCNConv,
    BatchNorm,
    global_mean_pool,
    global_max_pool,
    global_add_pool
)

/mnt/d/projects/wind_pressure_prediction_GNN/linux_wpp_GNN/lib/python3.12/site-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /mnt/d/projects/wind_pressure_prediction_GNN/linux_wpp_GNN/lib/python3.12/site-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/mnt/d/projects/wind_pressure_prediction_GNN/linux_wpp_GNN/lib/python3.12/site-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /mnt/d/projects/wind_pressure_prediction_GNN/linux_wpp_GNN/lib/python3.12/site-packages/torch_cluster/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/mnt/d/projects/wind_pressure_prediction_GNN/linux_wpp_GNN/lib/python3.12/site

In [2]:
SAVE_DIR = "/mnt/d/projects/wind_pressure_prediction_GNN/experiments/tuning_GNN"
LOG_CSV_PATH = "/mnt/d/projects/wind_pressure_prediction_GNN/experiments/log_gnn.csv"
train_graphs_dir = "/mnt/d/projects/wind_pressure_prediction_GNN/data/processed/Graphs_train"

In [3]:
def get_device():
    if torch.cuda.is_available():
        device = torch.device("cuda")
        print(f"🚀 Используется GPU: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device("cpu")
        print("💻 Используется CPU")
    return device

device = get_device()

💻 Используется CPU


In [4]:
# 🔧 Путь к графам трейна (для Linux / WSL)

def load_train_graphs(train_graphs_dir):
    """
    Загружает графы из указанной директории и возвращает список объектов Data (PyG)
    """
    graphs = []
    for filename in os.listdir(train_graphs_dir):
        if filename.endswith(".pt"):
            path = os.path.join(train_graphs_dir, filename)
            try:
                data = torch.load(path, weights_only=False)  # 👈 обязательно для PyTorch 2.6+
                graphs.append(data)
            except Exception as e:
                print(f"⚠️ Ошибка при загрузке {filename}: {e}")
    print(f"✅ Загружено {len(graphs)} графов из {train_graphs_dir}")
    return graphs

# 📦 Загрузка графов
train_graphs = load_train_graphs(train_graphs_dir)


✅ Загружено 486 графов из /mnt/d/projects/wind_pressure_prediction_GNN/data/processed/Graphs_train


In [5]:
class GCNRegressor(nn.Module):
    def __init__(
        self,
        in_channels,
        hidden_channels,
        out_channels,
        num_layers=3,
        activation_fn=nn.ReLU(),
        use_batchnorm=True,
        pooling='mean',  # 'mean', 'max', 'add'
        dropout=0.2
    ):
        super(GCNRegressor, self).__init__()

        self.activation_fn = activation_fn
        self.use_batchnorm = use_batchnorm
        self.dropout = dropout

        # 🔹 Слои GCN
        self.convs = nn.ModuleList()
        self.bns = nn.ModuleList()

        self.convs.append(GCNConv(in_channels, hidden_channels))
        if use_batchnorm:
            self.bns.append(BatchNorm(hidden_channels))

        for _ in range(num_layers - 1):
            self.convs.append(GCNConv(hidden_channels, hidden_channels))
            if use_batchnorm:
                self.bns.append(BatchNorm(hidden_channels))

        # 🔹 Пулинг
        if pooling == 'mean':
            self.pool = global_mean_pool
        elif pooling == 'max':
            self.pool = global_max_pool
        elif pooling == 'add':
            self.pool = global_add_pool
        else:
            raise ValueError(f"Unsupported pooling method: {pooling}")

        # 🔹 Финальный регрессор
        self.lin1 = nn.Linear(hidden_channels, hidden_channels)
        self.lin2 = nn.Linear(hidden_channels, out_channels)

    def forward(self, x, edge_index, batch):
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if self.use_batchnorm:
                x = self.bns[i](x)
            x = self.activation_fn(x)

        x = self.lin1(x)
        x = self.activation_fn(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lin2(x)
        return x

In [6]:
def evaluate_gnn_model(model, dataloader, device='cpu', save_dir=None, save_prefix="_eval"):
    model.eval()
    model.to(device)

    y_true_all = []
    y_pred_all = []

    with torch.no_grad():
        for batch in dataloader:
            batch = batch.to(device)

            outputs = model(batch.x, batch.edge_index, batch.batch)
            y_pred_all.extend(outputs.squeeze().cpu().numpy())
            y_true_all.extend(batch.y.squeeze().cpu().numpy())

    # 🧮 Метрики
    y_true = np.array(y_true_all)
    y_pred = np.array(y_pred_all)

            # 🔍 Проверка на NaN
    if np.isnan(y_true).any() or np.isnan(y_pred).any():
        print("❌ NaN обнаружен в evaluate_gnn_model!")
        print("🧪 y_true содержит NaN:", np.isnan(y_true).any())
        print("🧪 y_pred содержит NaN:", np.isnan(y_pred).any())
        print("🔍 Пример y_pred:", y_pred[:10])
        return np.nan, np.nan, np.nan  # или можно вернуть None, None, None

    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)

    return r2, mae, mse


In [7]:
def train_gnn_crossval(
    dataset,
    model_class,
    model_params,
    loss_fn,
    optimizer_class,
    optimizer_params,
    num_epochs,
    device,
    k_folds,
    batch_size,
    val_every,
    early_stopping_rounds,
    metric_for_early_stopping,
    maximize
):
    all_metrics = []

    kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

    for fold, (train_idx, val_idx) in enumerate(kfold.split(dataset)):
        print(f"\n🌀 Fold {fold + 1}/{k_folds}")

        train_subset = [dataset[i] for i in train_idx]
        val_subset = [dataset[i] for i in val_idx]

        train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

        model = model_class(**model_params).to(device)
        optimizer = optimizer_class(model.parameters(), **optimizer_params)

        best_metric = -np.inf if maximize else np.inf
        no_improve_epochs = 0

        for epoch in tqdm(range(1, num_epochs + 1), desc=f"🌀 Fold {fold + 1}"):
            model.train()
            total_loss = 0.0

            for batch in train_loader:
                batch = batch.to(device)
                optimizer.zero_grad()
                out = model(batch.x, batch.edge_index, batch.batch)
                loss = loss_fn(out.squeeze(), batch.y.squeeze())
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            avg_loss = total_loss / len(train_loader)

            # 🔄 Каждые val_every эпох + первая и последняя
            if epoch % val_every == 0 or epoch == 1 or epoch == num_epochs:
                model.eval()
                y_true_all = []
                y_pred_all = []

                with torch.no_grad():
                    for batch in val_loader:
                        batch = batch.to(device)
                        out = model(batch.x, batch.edge_index, batch.batch).squeeze()
                        y_true_all.append(batch.y.squeeze().cpu())
                        y_pred_all.append(out.cpu())

                y_true_all = torch.cat(y_true_all).numpy()
                y_pred_all = torch.cat(y_pred_all).numpy()

                                
                # 🔍 Проверка на NaN
                if np.isnan(y_true_all).any() or np.isnan(y_pred_all).any():
                    print("❌ NaN обнаружен в предсказаниях или истинных значениях!")
                    print("📌 Прерываем обучение на этом фолде.")
                    print("🧪 y_true_all содержит NaN:", np.isnan(y_true_all).any())
                    print("🧪 y_pred_all содержит NaN:", np.isnan(y_pred_all).any())
                    print("🔍 y_pred_all пример:", y_pred_all[:10])
                    return all_metrics  # можно заменить на break, если хочешь не выходить из функции


                r2 = r2_score(y_true_all, y_pred_all)
                mae = mean_absolute_error(y_true_all, y_pred_all)
                mse = mean_squared_error(y_true_all, y_pred_all)

                print(f"📉 Epoch {epoch:02d} | Train Loss: {avg_loss:.6f} | "
                    f"Val R²: {r2:.4f} | "
                    f"MAE: {mae:.4f} | "
                    f"MSE: {mse:.6f}")


            # Валидация
            if epoch % val_every == 0 or epoch == num_epochs:
                r2, mae, mse = evaluate_gnn_model(model, val_loader, device=device, save_dir=None)

                current_metric = {"r2": r2, "mae": mae, "mse": mse}[metric_for_early_stopping]

                is_better = (current_metric > best_metric) if maximize else (current_metric < best_metric)

                if is_better:
                    best_metric = current_metric
                    no_improve_epochs = 0
                else:
                    no_improve_epochs += 1
                    print(f"⏸️  No improvement for {no_improve_epochs} validation checks")
         
                if no_improve_epochs >= early_stopping_rounds:
                    print(f"🛑 Early stopping at epoch {epoch}")
                    print("📊 Final Evaluation on Validation Set:")
                    print(f"   R²   = {r2:.4f}")
                    print(f"   MAE  = {mae:.4f}")
                    print(f"   MSE  = {mse:.6f}")
                    break
    

        all_metrics.append({
            "fold": fold + 1,
            "r2": r2,
            "mae": mae,
            "mse": mse
        })

    # 📊 Средние метрики по всем фолдам
    mean_r2 = np.mean([m["r2"] for m in all_metrics])
    mean_mae = np.mean([m["mae"] for m in all_metrics])
    mean_mse = np.mean([m["mse"] for m in all_metrics])

    return all_metrics


In [8]:
def create_run_directory(run_name="gnn"):
    base_dir = SAVE_DIR
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    run_id = f"{run_name}_{timestamp}"
    run_dir = os.path.join(base_dir, run_id)
    os.makedirs(run_dir, exist_ok=True)
    return run_id, run_dir


def save_gnn_run_log(run_id, run_dir, config, avg_metrics, log_csv_path, model=None, r2_threshold=0.2):
    """
    Сохраняет конфигурацию, метрики и веса модели, если достигнут порог R².
    """
    # ➤ Сохраняем config
    config_path = os.path.join(run_dir, f"config_{run_id}.json")
    with open(config_path, "w") as f:
        json.dump(config, f, indent=4)

    # ➤ Готовим лог
    log_header = "run_id,model,lr,batch_size,activation_fn,optimizer,hidden_channels,num_layers,use_batchnorm,pooling,dropout,R2,MAE,MSE\n"
    log_line = (
        f"{run_id},gnn,{config['lr']},{config['batch_size']},"
        f"{config['activation_fn']},{config['optimizer']},"
        f"{config['hidden_channels']},{config['num_layers']},{config['use_batchnorm']},"
        f"{config['pooling']},{config['dropout']},"
        f"{avg_metrics['r2']:.4f},{avg_metrics['mae']:.4f},{avg_metrics['mse']:.6f}\n"
    )

    # ➤ Записываем лог
    write_header = not os.path.exists(log_csv_path)
    with open(log_csv_path, "a") as f:
        if write_header:
            f.write(log_header)
        f.write(log_line)

    return config_path, log_csv_path


In [9]:
def save_gnn_run_log(run_id, run_dir, config, avg_metrics, log_csv_path, model=None):
    # 📁 Сохраняем конфиг и метрики в run_dir (если нужно)
    os.makedirs(run_dir, exist_ok=True)
    
    # 💾 Формирование одной строки для CSV
    row = {
        "trial": run_id,
        "r2": avg_metrics["r2"],
        "mae": avg_metrics["mae"],
        "mse": avg_metrics["mse"],
        "hidden_channels": config["hidden_channels"],
        "batch_size": config["batch_size"],
        "num_layers": config["num_layers"],
        "activation_fn": config["activation_fn"],
        "use_batchnorm": config["use_batchnorm"],
        "pooling": config["pooling"],
        "dropout": config["dropout"],
        "lr": config["lr"],
        "optimizer": config["optimizer"]
    }

    # 🧾 Запись в лог CSV
    csv_exists = os.path.isfile(log_csv_path)
    with open(log_csv_path, mode='a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=list(row.keys()))
        if not csv_exists:
            writer.writeheader()
        writer.writerow(row)

In [10]:
EPOCHS = 200

def run_gnn_random_search_and_train(
    dataset,
    num_trials,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    num_epochs=EPOCHS,
    k_folds=5,
    val_every=10
    ):
    param_space = {
        "hidden_channels": [64, 128],
        "batch_size": [16, 32],
        "num_layers": [4, 5, 6, 8, 10],
        "activation_fn": [nn.ReLU(), nn.ELU(), nn.LeakyReLU(0.1)],
        "use_batchnorm": [True, False],
        "pooling": ["mean", "max", "add"],
        "dropout": [0.0, 0.1, 0.2, 0.3],
        "lr": [0.001, 0.0005, 0.0001, 0.00075, 0.00025],
        "optimizer": ["Adam"]
    }

    best_r2 = -float("inf")
    best_config = None
    best_metrics = None
    log = []

    for trial in range(num_trials):
        print(f"\n🎲 Random Search Trial {trial + 1}/{num_trials}")

        config = {
            "hidden_channels": random.choice(param_space["hidden_channels"]),
            "batch_size": random.choice(param_space["batch_size"]),
            "num_layers": random.choice(param_space["num_layers"]),
            "activation_fn": random.choice(param_space["activation_fn"]),
            "use_batchnorm": random.choice(param_space["use_batchnorm"]),
            "pooling": random.choice(param_space["pooling"]),
            "dropout": random.choice(param_space["dropout"]),
            "lr": random.choice(param_space["lr"]),
            "optimizer": random.choice(param_space["optimizer"]),
        }

        print("🛠 Гиперпараметры:")
        for k, v in config.items():
            name = v.__class__.__name__ if callable(v) else v
            print(f"   {k:<15}: {name}")


        model_params = {
            "in_channels": 5,
            "hidden_channels": config["hidden_channels"],
            "out_channels": 1,
            "num_layers": config["num_layers"],
            "activation_fn": config["activation_fn"],
            "use_batchnorm": config["use_batchnorm"],
            "pooling": config["pooling"],
            "dropout": config["dropout"]
        }


        optimizer_class = torch.optim.Adam if config["optimizer"] == "Adam" else torch.optim.SGD
        optimizer_params = {"lr": config["lr"]}
        if config["optimizer"] == "SGD":
            optimizer_params["momentum"] = 0.9

        metrics = train_gnn_crossval(
            dataset=deepcopy(dataset),
            model_class=GCNRegressor,
            model_params=model_params,
            loss_fn=nn.MSELoss(),
            optimizer_class=optimizer_class,
            optimizer_params=optimizer_params,
            device=device,
            num_epochs=num_epochs,
            k_folds=k_folds,
            val_every=val_every,
            batch_size=config["batch_size"],  # ← ЭТО ДОБАВЬ
            early_stopping_rounds=10,               # ← ОБЯЗАТЕЛЕН
            metric_for_early_stopping="r2",         # ← ОБЯЗАТЕЛЕН
            maximize=True 
        )

        avg_metrics = {
            "r2": np.mean([m["r2"] for m in metrics]),
            "mae": np.mean([m["mae"] for m in metrics]),
            "mse": np.mean([m["mse"] for m in metrics]),
        }

        print()
        print(f"✅ Trial {trial + 1} — Средние метрики:")
        for k, v in avg_metrics.items():
            print(f"   {k.upper():<5}: {v:.4f}")

        log.append({
            "trial": trial + 1,
            "config": {k: (v.__class__.__name__ if callable(v) else v) for k, v in config.items()},
            "metrics": avg_metrics
        })

        run_id, run_dir = create_run_directory("gnn")

        config_serializable = {
            k: v.__class__.__name__ if callable(v) else v
            for k, v in config.items()
        }

        save_gnn_run_log(
            run_id=run_id,
            run_dir=run_dir,
            config=config_serializable,  # ✅ безопасная версия
            avg_metrics=avg_metrics,
            log_csv_path=LOG_CSV_PATH,
            model=None
        )

        if avg_metrics["r2"] > best_r2:
            best_r2 = avg_metrics["r2"]
            best_config = config
            best_metrics = metrics

    print("\n📜 Лог всех прогонов:")
    for entry in log:
        print(f"Trial {entry['trial']}:")
        print(f"   R²   : {entry['metrics']['r2']:.4f}")
        print(f"   MAE  : {entry['metrics']['mae']:.4f}")
        print(f"   MSE  : {entry['metrics']['mse']:.6f}")
        print(f"   🔧 Гиперпараметры:")
        for k, v in entry['config'].items():
            print(f"      {k:<15}: {v}")


    return best_config, best_metrics, log

In [11]:
if __name__ == "__main__":
    dataset = load_train_graphs(train_graphs_dir)
    best_config, best_metrics, log = run_gnn_random_search_and_train(
        dataset,
        num_trials=150,
        )




✅ Загружено 486 графов из /mnt/d/projects/wind_pressure_prediction_GNN/data/processed/Graphs_train

🎲 Random Search Trial 1/150
🛠 Гиперпараметры:
   hidden_channels: 128
   batch_size     : 16
   num_layers     : 10
   activation_fn  : LeakyReLU
   use_batchnorm  : False
   pooling        : add
   dropout        : 0.0
   lr             : 0.001
   optimizer      : Adam

🌀 Fold 1/5


/mnt/d/projects/wind_pressure_prediction_GNN/linux_wpp_GNN/lib/python3.12/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
🌀 Fold 1:   0%|          | 1/200 [00:04<13:33,  4.09s/it]

📉 Epoch 01 | Train Loss: 0.251060 | Val R²: 0.1864 | MAE: 0.3395 | MSE: 0.198017


🌀 Fold 1:   4%|▍         | 9/200 [00:31<10:47,  3.39s/it]

📉 Epoch 10 | Train Loss: 0.088838 | Val R²: 0.5739 | MAE: 0.2272 | MSE: 0.103701


...

📉 Epoch 10 | Train Loss: 0.135527 | Val R²: 0.4505 | MAE: 0.2902 | MSE: 0.146225


🌀 Fold 3:  10%|▉         | 19/200 [00:40<06:41,  2.22s/it]

📉 Epoch 20 | Train Loss: 0.094748 | Val R²: 0.6264 | MAE: 0.2352 | MSE: 0.099431


🌀 Fold 3:  11%|█         | 22/200 [00:48<06:34,  2.22s/it]


KeyboardInterrupt: 